<a href="https://colab.research.google.com/github/josha106/Josha/blob/main/dacon_%EC%9E%AC%EC%A0%95%EC%A0%95%EB%B3%B4%EB%8C%80%ED%9A%8C_sota%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb peft
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-gpu

!pip install rank_bm25 langchain-teddynote pymupdf4llm

In [ ]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
# import fitz

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator


from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain.vectorstores import FAISS

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# new added
from langchain.text_splitter import MarkdownTextSplitter, MarkdownHeaderTextSplitter
import pymupdf4llm
from langchain_teddynote.retrievers import KiwiBM25Retriever

In [ ]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    chunks_list=[]
    # PDF 파일 열기
    markdown = pymupdf4llm.to_markdown(file_path)
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    markdown_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    markdown_chunks = markdown_header_splitter.split_text(markdown)

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(markdown_chunks)

    return chunks

def create_vector_db(chunks, model_path="intfloat/multilingual-e5-large"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성 ie. /content/drive/MyDrive/open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path, chunk_size=400)
        db = create_vector_db(chunks)

        # Retriever 생성
        faiss = db.as_retriever()
        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)

        # Retriever Ensemble
        retriever = EnsembleRetriever(
                                    retrievers=[kiwi_bm25_retriever, faiss],  # 사용할 검색 모델의 리스트
                                    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
                                    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
                                    )

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# DB생성
import pandas as pd
base_directory = '/content/drive/MyDrive/Colab Notebooks/dacon/재정정보/open_data/' # Your Base Directory
df = pd.read_csv('/content/drive/MyDrive/open/test.csv')

In [ ]:
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 국토교통부_전세임대(융자)...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

...

Processing PDFs:  78%|███████▊  | 7/9 [01:45<00:29, 14.58s/it]

Processing 국토교통부_민간임대(융자)...


Processing PDFs:  89%|████████▉ | 8/9 [01:57<00:13, 13.85s/it]

Processing 고용노동부_조기재취업수당...


Processing PDFs: 100%|██████████| 9/9 [02:10<00:00, 14.47s/it]


In [ ]:
# MODEL Import

def setup_llm_pipeline():
    # 4비트 양자화 설정 sota: byung-jun/EEVE-Korean-Instruct-10.8B-v1.0_trained_v1
    # yanolja/EEVE-Korean-Instruct-10.8B-v1.0
    model_id = "byung-jun/EEVE-Korean-Instruct-10.8B-v1.0_trained_v1"
    tokenizer_id = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_id,  eos_token='</s>')
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                quantization_config=bnb_config,
                                                device_map={"":0}, # gpu에 올림.
                                                #torch_dtype=torch.float32,
                                                )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

# LLM 파이프라인
llm = setup_llm_pipeline()

# Langchain을 이용한 추론
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문:
    {question}

    질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })



tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

...

Answer: 고용보험사업의 사업시행주체는 고용노동부(고용센터)입니다.

-----  
① 사업추진체계  
-  사업시행방법 : 직접수행  
-  사업시행주체 : 고용노동부(고용센터)  
-  사업 수혜자 : 구직급여 수혜자  
-  보조, 융자, 출연, 출자 등의 경우 보조․융자 등 지원 비율 및 법적근거: 해당없음  
-  ‘93.12월 고용보험법 제정․

Question: 사업 집행절차에서 조기재취업수당을 받기 위해 필요한 절차는 무엇인가요?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:  99%|█████████▊| 78/79 [1:15:04<00:53, 53.20s/it]

Answer: 조기재취업수당 신청서 제출 → 지급요건 확인 → 조기재취업수당 지급  
-----  
|Col1|Col2|Col3|
|---|---|---|
|사업명|프로그램명|단위사업명|
|조기재취업수당|조기재취업수당|조기재취업수당|
|구직급여 수급자가 조기 재취업하면 조기재취업수당 등을 지급하여 구직급여 수급자의 장기실업 방지 및 조기 재취업 촉진|구직급여 수급

Question: 직업능력개발 수당을 받기 위해 어떤 절차를 거쳐야 하나요?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions: 100%|██████████| 79/79 [1:16:01<00:00, 57.74s/it]

Answer: 직업능력개발 수당을 받기 위해서는 직업능력개발 훈련을 받고자 하는 경우 직업안정기관의 장이 지정한 훈련기관에 훈련을 신청하고, 훈련을 수료한 후 직업안정기관의 장에게 훈련 수당을 신청해야 합니다.

-----  
**고용보험법 제65조(직업능력개발 수당) ①직업능력개발 수당은 수급자격자가 직업안정기관의 장이 지정한 훈련기관에서 직업능력개발 훈련을 받는 경우에 그 직업능력개발 훈련을



In [ ]:
def sub_preprocessing(df, answer_name):
  df[f'{answer_name}_pre'] = df[answer_name].apply(lambda x : x.split('\n\n')[0])
  df.drop(answer_name, axis=1, inplace=True)
  df.rename(columns={f'{answer_name}_pre':answer_name}, inplace=True)
  return df

In [ ]:
# 생성된 답변을 제출 DataFrame에 추가
train_df = df.copy()
train_df['model_Answer'] = [item['Answer'] for item in results]
train_df['model_Answer'] = train_df['model_Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]
train_df = sub_preprocessing(train_df, 'model_Answer')

train_df.to_csv("submission.csv", encoding='UTF-8-sig', index=False)

8:16sotawithmd.csv: 0.6277295511